In [1]:
import seaborn as sn
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from datetime import datetime

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models import LinearAxis,  Range1d

import numpy as np

%matplotlib inline
output_notebook()

Loading BokehJS ...

In [2]:

def read_files(filenames):
    dfs = []
    for filename in filenames:
        time = datetime.fromtimestamp(os.path.getmtime(filename)).strftime('%Y-%m-%d %H:%M:%S')
        dfs.append([os.path.getmtime(filename),os.path.getsize(filename),os.path.basename(filename)])

    # take second element for sort
    def takefirst(elem):
        return elem[0]

    # sort list with key
    dfs.sort(key=takefirst)

    result = pd.DataFrame.from_records(dfs)
    result.columns = ['Time','Size', 'File']
    result.Size = result.Size / (1024**2)
    result['Change'] = result.Size.diff()
    result['Rolling'] = result.Change.rolling(5).mean()
    
    return result


In [3]:

def generate_graph(df):

    source = ColumnDataSource(df)

    p = figure(width=750, height=500)

    p.vbar(x ='index', top='Size', width=0.9, source=source, alpha=0.5)

    p.xgrid.grid_line_color = None
    p.y_range.start = 0

    p.title.text = 'Growth of Path File over Simruns'
    p.xaxis.axis_label = 'Simulation Run'
    p.yaxis.axis_label = 'File Size'

    # Setting the second y axis range name and range
    p.extra_y_ranges = {"foo": Range1d(start=0, end=5)}

    # Adding the second axis to the plot.  
    p.add_layout(LinearAxis(y_range_name="foo"), 'right')

    p.line(x ='index', y='Change', color="red", line_width=1, source=source, alpha=0.8, y_range_name="foo")
    p.line(x ='index', y='Rolling', color="green", line_width=2, source=source, alpha=1, y_range_name="foo")
    
    hover = HoverTool()
    hover.tooltips=[
        ('File', '@File'),
        ('Size', '@Size'),
        ('Delta', '@Change'),
        ('Time', '@Time')
    ]

    p.add_tools(hover)
    
    show(p)
    

In [7]:
path = 'C:/Users/shafeeq.mollagee/OneDrive - Aurecon Group/GIPTN Traffic Modelling/04 - CBD Modelling/08 - Micro Model/03 - Final Base Model/Rev 08/Scenarios/S000001/Paths and Costs'
filenames = glob.glob(path + "/*.weg")

generate_graph(read_files(filenames))

output_file('C:/Users/shafeeq.mollagee/OneDrive - Aurecon Group/GIPTN Traffic Modelling/04 - CBD Modelling/08 - Micro Model/03 - Final Base Model/Rev 08/Scenarios/S000001/Paths and Costs/testPaths4.html')
